In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
from collections import Counter

from tqdm import tqdm
import azure.cosmos.cosmos_client as cosmos_client

# Useful queries

Example queries for the MegaDB's `sequences` table. MegaDB is a NoSQL database on Azure Cosmos DB.

Cosmos DB Python SDK documentation: https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python

Resource URI Syntax for REST: https://docs.microsoft.com/en-us/rest/api/cosmos-db/cosmosdb-resource-uri-syntax-for-rest

Subquery for nested objects: https://docs.microsoft.com/en-us/azure/cosmos-db/sql-query-subquery

## Connect to the Cosmos DB instance

`COSMOS_ENDPOINT` and `COSMOS_KEY` need to be environment variables. 

In [5]:
# Cosmos DB config
config = {
    'ENDPOINT': os.environ.get('COSMOS_ENDPOINT'),
    'PRIMARYKEY': os.environ.get('COSMOS_KEY')
}

# Initialize the Cosmos client
client = cosmos_client.CosmosClient(url_connection=config['ENDPOINT'], auth={
                                    'masterKey': config['PRIMARYKEY']})

sequences_table = 'dbs/camera-trap/colls/sequences'  # database link + container link
datasets_table = 'dbs/camera-trap/colls/datasets'

options = {
    'enableCrossPartitionQuery': True
}

### Get the `datasets` table

which records the location and access levels of each dataset.

In [21]:
%%time

query = {'query': '''SELECT * FROM datasets d'''}

result_iterable = client.QueryItems(datasets_table, query, options)

datasets = {i['dataset_name']:{k: v for k, v in i.items() if not k.startswith('_')} for i in iter(result_iterable)}

print('Length of results:', len(datasets))

Length of results: 18
CPU times: user 6.37 ms, sys: 2.21 ms, total: 8.59 ms
Wall time: 149 ms


#### List the public datasets

In [24]:
public_datasets = [d['dataset_name'] for d in datasets.values() if 'public' in d['access']]
public_datasets

['wcs', 'bellevue_190602', 'caltech', 'nacti']

## Examples

In the examples, we limit the selection to a few entries using the `TOP` keyword. When using the DB to create datasets, delete the TOP keyword and arg.

Specifying the `partition_key` doesn't seem to work for restricting the query to one dataset...

In [52]:
options = {
    'enableCrossPartitionQuery': True
}

### What datasets are there that have sequences data?

In [10]:
%%time

query = {'query': '''
SELECT DISTINCT seq.dataset
FROM sequences seq
'''}

result_iterable = client.QueryItems(sequences_table, query, options)

results = []
for item in iter(result_iterable):
    res = item['dataset']
    results.append(res)

print('Length of results:', len(results))

Length of results: 6
CPU times: user 25.3 ms, sys: 3.18 ms, total: 28.5 ms
Wall time: 11.5 s


In [11]:
results

['peaceparks_201908_humans',
 'wps_190624',
 'wps_190624',
 'zsl_borneo',
 'zsl_borneo',
 'zsl_borneo']

In [12]:
set(results)

{'peaceparks_201908_humans', 'wps_190624', 'zsl_borneo'}

### Image entries in a dataset with class "empty" - demonstrating JOIN
Can use `partition_key` for this query without needing joins

Refer to https://docs.microsoft.com/en-us/azure/cosmos-db/sql-query-join - query is constructed as follows (thanks to Chris Ritchie):
1. Iterate through each document (seq) (query scoped to a single partition otherwise you could add an additional WHERE clause for seq.dataset = 'zsl_borneo'): `FROM sequences seq`
2. For each document (seq) expand each child element in the images array (im): `im IN seq.images`
3. Apply a cross product with the root of the item (seq) with each child element (im) the second step flattened
4. Project each child element (im): `JOIN`

In [97]:
%%time

dataset_name = 'zsl_borneo'

query = {'query': '''
SELECT TOP 10 im.file, im.class, seq.seq_id
FROM sequences seq JOIN im IN seq.images 
WHERE seq.dataset = "{}" 
    AND ARRAY_LENGTH(im.class) > 0 
    AND ARRAY_CONTAINS(im.class, "empty")
'''.format(dataset_name)}

# WHERE ARRAY_LENGTH(im.class) > 0 AND ARRAY_CONTAINS(im.class, "empty")

result_iterable = client.QueryItems(sequences_table, query, options)

results = [item for item in iter(result_iterable)]

print('Length of results:', len(results))

Length of results: 10
CPU times: user 4.58 ms, sys: 1.48 ms, total: 6.07 ms
Wall time: 82.8 ms


### All images with bounding box annotation

In [50]:
%%time

query = {'query': '''
SELECT TOP 10 im.bbox, im.file
FROM im IN sequences.images 
WHERE ARRAY_LENGTH(im.bbox) > 0
'''}

result_iterable = client.QueryItems(sequences_table, query, options, partition_key='zsl_borneo')
# if you want to restrict to one dataset, pass in partition_key=dataset - doesn't work here?

results = [item for item in iter(result_iterable)]

print('Length of results:', len(results))

Length of results: 10
CPU times: user 9.41 ms, sys: 2.11 ms, total: 11.5 ms
Wall time: 157 ms


### All images with the specified species at the image level

In [55]:
%%time

species_requested = 'horse-tailed squirrel'

query = {'query': '''
SELECT TOP 10 im.class, im.file
FROM im IN sequences.images 
WHERE ARRAY_LENGTH(im.class) > 0 AND ARRAY_CONTAINS(im.class, "{}")
'''.format(species_requested)}

result_iterable = client.QueryItems(sequences_table, query, options)

results = [item for item in iter(result_iterable)]

print('Length of results:', len(results))

Length of results: 10
CPU times: user 7.48 ms, sys: 1.88 ms, total: 9.37 ms
Wall time: 423 ms


### Species count where the label is at the sequence level

In [80]:
%%time

query = {'query': '''
SELECT TOP 100 seq.class
FROM sequences seq
WHERE ARRAY_LENGTH(seq.class) > 0
'''}

result_iterable = client.QueryItems(sequences_table, query, options)

species = Counter()
for item in iter(result_iterable):
    res = item['class']
    species.update(res)

CPU times: user 26.3 ms, sys: 3.77 ms, total: 30 ms
Wall time: 838 ms


In [81]:
species

Counter({'human': 100})

### Species count where the label is at the image level

In [66]:
%%time

query = {'query': '''
SELECT TOP 100000 im.class
FROM im IN sequences.images 
WHERE ARRAY_LENGTH(im.class) > 0
'''}

result_iterable = client.QueryItems(sequences_table, query, options)

species = Counter()
for item in iter(result_iterable):
    res = item['class']
    species.update(res)

CPU times: user 6.01 s, sys: 378 ms, total: 6.38 s
Wall time: 2min 52s


We can increase the throughput of the database when doing large queries to get response faster.

In [67]:
species

Counter({'__label_unavailable': 88489,
         'banded civet': 307,
         'banded linsang': 2,
         'bearded pig': 271,
         'binturong': 7,
         'black-capped babbler': 5,
         'blue-headed pitta': 10,
         'bornean banded pitta': 9,
         'bornean ground-babbler': 10,
         "bulwer's pheasant": 50,
         'emerald dove': 10,
         'empty': 3473,
         'great argus': 286,
         'greater mouse-deer': 2711,
         'horse-tailed squirrel': 32,
         'human': 810,
         'lesser mouse-deer': 135,
         'long-tailed macaque': 23,
         'long-tailed porcupine': 74,
         "low's squirrel": 22,
         'malay civet': 94,
         'malay porcupine': 116,
         'plain treeshrew': 3,
         'red muntjac': 1528,
         'sambar deer': 33,
         'short-tailed mongoose': 40,
         'southern pig-tailed macaque': 667,
         'spiny rat': 264,
         'thick-spined porcupine': 39,
         'yellow muntjac': 478,
         'yellow-

### Total number of sequence entries in database

In [72]:
%%time

query = {'query': '''
SELECT VALUE COUNT(1)
FROM seq
'''}

result_iterable = client.QueryItems(sequences_table, query, options)

for seq_count in iter(result_iterable):
    print(seq_count)

146526
CPU times: user 23.2 ms, sys: 3.62 ms, total: 26.9 ms
Wall time: 895 ms


### Total number of images in all sequences

In [73]:
%%time

query = {'query': '''
SELECT VALUE COUNT(1)
FROM im IN sequences.images 
'''}

result_iterable = client.QueryItems(sequences_table, query, options)

for im_count in iter(result_iterable):
    print(im_count)

596773
CPU times: user 26.7 ms, sys: 4.01 ms, total: 30.7 ms
Wall time: 12 s


### List last inserted sequences based on insertion timestap

In [77]:
%%time

query = {'query': '''
SELECT TOP 10 seq.dataset, seq._ts, seq.seq_id
FROM sequences seq
ORDER BY seq._ts DESC
'''}

result_iterable = client.QueryItems(sequences_table, query, options)

results = [item for item in iter(result_iterable)]

CPU times: user 8.21 ms, sys: 1.88 ms, total: 10.1 ms
Wall time: 163 ms


In [78]:
results

[{'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50869'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50870'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50871'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50872'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50873'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50874'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50875'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50876'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50877'},
 {'_ts': 1573253030, 'dataset': 'zsl_borneo', 'seq_id': '50878'}]